In [5]:
import pandas as pd
import json
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
print(pd.__version__)
df = pd.read_json('radical/analyse_fase-kenmerken_onderzoek/Data/bs-posts-2-clean.jsonl.gz', orient='records', lines=True, compression='gzip')
df.head()





3.0.0


,uri,text,likes,replies,reposts,quotes,scraped_at_local_time,account,posted_at
0,at://did:plc:zg3mbvhajrmif7g2lzuzwtta/app.bsky...,Here's staff on an ICE deportation flight givi...,240,3,60,2,2026-02-07 19:17:47.663,ottergoose.net,2026-01-19 17:12:47.904
1,at://did:plc:ako55s6m4irasu37djcwhtgp/app.bsky...,I saw a video of an ICE agent with an accent e...,58,1,2,0,2026-02-07 19:17:47.663,g4techtalker.bsky.social,2026-01-19 22:53:55.180
2,at://did:plc:qtjpurf5fsptp3dhi52kksl6/app.bsky...,"When ICE comes to the neighborhood, communitie...",51,1,21,1,2026-02-07 19:17:47.663,thenation.com,2026-01-19 13:21:27.853
3,at://did:plc:ysdibsjglggly2lpmoekupxd/app.bsky...,"On January 23rd, we take to the streets to cha...",36,2,16,0,2026-02-07 19:17:47.663,psc-cuny.org,2026-01-19 16:32:26.983
4,at://did:plc:xthfrsjmikovkc2gtfw6n7fg/app.bsky...,"Your daily reminder that, besides many seeming...",59,2,18,0,2026-02-07 19:17:47.663,jonbowzerbauman.bsky.social,2026-01-19 18:10:02.814


In [6]:
model_name = "tabularisai/multilingual-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Loading weights:   0%|          | 0/104 [00:00<?, ?it/s]

In [7]:

def predict_in_batches(texts, batch_size=32):
    all_labels = []
    all_confidences = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]

        inputs = tokenizer(
            batch,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        )

        with torch.no_grad():
            outputs = model(**inputs)

        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

        sentiment_map = {
            0: "Very Negative",
            1: "Negative",
            2: "Neutral",
            3: "Positive",
            4: "Very Positive"
        }

        for p in probs:
            idx = torch.argmax(p).item()
            all_labels.append(sentiment_map[idx])
            all_confidences.append(float(p[idx]))

    return all_labels, all_confidences


In [ ]:
labels, confs = predict_in_batches(df["text"].astype(str).tolist(), batch_size=16)

df["sentiment"] = labels
df["confidence"] = confs

#exporteren van gelabelde dataset
# df.to_json('radical/analyse_fase-kenmerken_onderzoek/Data/bs-posts2-sentiment-labeld.jsonl.gz', orient='records', lines=True, compression='gzip')

/tmp/ipykernel_89937/2652681866.py:7: Pandas4Warning: The default 'epoch' date format is deprecated and will be removed in a future version, please use 'iso' date format instead.
  df.to_json('radical/analyse_fase-kenmerken_onderzoek/Data/bs-posts2-sentiment-labeld.jsonl.gz', orient='records', lines=True, compression='gzip')


In [9]:
df['sentiment'].value_counts()

sentiment
Neutral          140353
Negative          45864
Very Negative     37778
Very Positive     10593
Positive           8530
Name: count, dtype: int64